# Reading input data and building the SFS

The first step when running delimitpy is to read the input alignments and use them to construct a site frequency spectrum (SFS).

In [1]:
from delimitpy import parse_input
from delimitpy import process_empirical
import numpy as np
import os
import pickle

# Read in intermediate files from previous part of tutorial.


In [2]:
config_parser = parse_input.ModelConfigParser("../../examples/test1/config.txt")
config_values = config_parser.parse_config()
labels = np.load(os.path.join(config_values["output directory"], 'labels.npy'), allow_pickle=True)
with open(os.path.join(config_values["output directory"], 'parameterized_models.pickle'), 'rb') as f:
    parameterized_models = pickle.load(f)


In [6]:
data_processor = process_empirical.DataProcessor(parameterized_models, config=config_values)
empirical_array = data_processor.fasta_to_numpy()
print(empirical_array.shape)
empirical_2d_sfs_sampling = data_processor.find_downsampling(empirical_array)
print(empirical_2d_sfs_sampling[(6,4,5)])



(30, 21548)
updated
(30, 1038)
finding projection values
1038


In [9]:
empirical_2d_sfs = data_processor.numpy_to_2d_sfs(empirical_array, downsampling={"A":6, "B":4, "C":5}, replicates = 10)
empirical_msfs = data_processor.numpy_to_msfs(empirical_array, downsampling={"A":6, "B": 4, "C":5}, replicates = 10)


build sfs
